In [1]:
import re
import os
import time
import random
import requests
import numpy as np
import pandas as pd
from os import system
from math import floor
from copy import deepcopy
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
criteria_table = pd.read_excel('Data_for_Scraping_Electrician.xlsx',sheet_name='Criterias')

In [3]:
driver = webdriver.Chrome()

In [4]:
driver.get("http://www.aef.cci.fr/rechercheMulticritere")

In [25]:
criteria_table.head()

,A - Commune,B - NAF 2008,C - Effectif
0,Aix-en-Provence,4321A,0 salarie
1,Allauch,NaN,1 - 2 salaries
2,Alleins,NaN,3 - 5 salaries
3,Arles,NaN,6 - 9 salaries
4,Aubagne,NaN,10 - 19 salaries


In [30]:
criteria_table['C - Effectif'].value_counts()

10 - 19 salaries    1
6 - 9 salaries      1
3 - 5 salaries      1
20 - 49 salaries    1
0 salarie           1
1 - 2 salaries      1
Name: C - Effectif, dtype: int64

In [8]:
A_list = criteria_table['A - Commune']

In [37]:
A_list[:5]

0    Aix-en-Provence
1            Allauch
2            Alleins
3              Arles
4            Aubagne
Name: A - Commune, dtype: object

In [38]:
commune = driver.find_element_by_xpath('/html/body/div[3]/div[6]/div[1]/div[2]/div/div/div[1]/dl[1]/dd/ol/li[4]/a')
searchbox = driver.find_element_by_xpath('/html/body/div[3]/div[6]/div[1]/div[2]/div/div/form/ul/li[2]/fieldset[4]/p[1]/input')


In [35]:
commune.click()

In [91]:
driver.get("http://www.aef.cci.fr/rechercheMulticritere")

In [105]:
for i in A_list[:5]:
    print(i)

Aix-en-Provence
Allauch
Alleins
Arles
Aubagne


In [6]:
C_list = ['0 salarie', '1 - 2 salaries', '3 - 5 salaries', '6 - 9 salaries', '10 - 19 salaries', '20 - 49 salaries']

In [187]:
driver = webdriver.Chrome()
driver.get("http://www.aef.cci.fr/rechercheMulticritere")

In [185]:
len(set(url_list))

69

In [186]:
url_lst = []
for i in A_list:
    for j in C_list:
        try:
            driver.get("http://www.aef.cci.fr/rechercheMulticritere")
            time.sleep(3)
            driver.find_element_by_xpath("//a[contains(text(), 'Commune')]").click()
            searchbox = driver.find_element_by_id("filtreLocalisationCommunes").send_keys(i)
            time.sleep(1)
            try:
                time.sleep(1)
                driver.find_element_by_css_selector("[title*={}]".format(i)).click()
            except:
                driver.find_element_by_id("filtreLocalisationCommunes").send_keys('a')
                time.sleep(0.5)
                driver.find_element_by_id("filtreLocalisationCommunes").send_keys(Keys.BACKSPACE)
                time.sleep(0.5)
                driver.find_element_by_css_selector("[title*={}]".format(i)).click()

            time.sleep(1)
            driver.find_element_by_xpath('//a[@href="'+'javascript:page.transferer()'+'"]').click()

            #actives
            driver.find_element_by_id('activites').click()
            driver.find_element_by_xpath('//*[@id="barreActivites"]/ol/li[5]/a').click()
            searchbox_b = driver.find_element_by_xpath('//*[@id="filtreActivitesSousClassesNAF2008"]').send_keys("4321A")
            time.sleep(1)
            driver.find_element_by_xpath('//*[@id="transfertComplet"]/img').click()

            driver.find_element_by_xpath('//*[@id="donneesChiffrees"]').click()
            time.sleep(3)
            driver.find_element_by_xpath('//*[@title="{}"]'.format(j)).click()
            driver.find_element_by_xpath('//a[img/@src="images/b_fleche.gif"]').click()
            driver.find_element_by_id('bComptage').click()
            time.sleep(3)
            driver.find_element_by_class_name('boutonSuivant').click()
            time.sleep(3)
            for i in range(1, len(driver.find_elements_by_xpath('//*[@id="listeEntreprises"]/li/table/tbody/tr'))+1):
                driver.find_element_by_xpath('//*[@id="listeEntreprises"]/li/table/tbody/tr[{}]/td[1]/strong/a'.format(i)).click()
                url_list.append(driver.current_url)
                driver.back()
                time.sleep(1)
                for i in range(3, len(driver.find_elements_by_xpath('//*[@id="listePages"]/li'))):
                    try:
                        driver.find_element_by_link_text('Page suivante').click()
                    except:
                        done = True
                        break
                    for i in range(1, len(driver.find_elements_by_xpath('//*[@id="listeEntreprises"]/li/table/tbody/tr'))+1):
                        driver.find_element_by_xpath('//*[@id="listeEntreprises"]/li/table/tbody/tr[{}]/td[1]/strong/a'.format(i)).click()
                        url_list.append(driver.current_url)
                        driver.back()          


            

        except:
            break

In [168]:
url_list = []
done = False
while done == False:
    for i in range(1, len(driver.find_elements_by_xpath('//*[@id="listeEntreprises"]/li/table/tbody/tr'))+1):
            driver.find_element_by_xpath('//*[@id="listeEntreprises"]/li/table/tbody/tr[{}]/td[1]/strong/a'.format(i)).click()
            url_list.append(driver.current_url)
            driver.back()
            time.sleep(1)
    for i in range(3, len(driver.find_elements_by_xpath('//*[@id="listePages"]/li'))):
        try:
            driver.find_element_by_link_text('Page suivante').click()
        except:
            done = True
            break
        for i in range(1, len(driver.find_elements_by_xpath('//*[@id="listeEntreprises"]/li/table/tbody/tr'))+1):
            driver.find_element_by_xpath('//*[@id="listeEntreprises"]/li/table/tbody/tr[{}]/td[1]/strong/a'.format(i)).click()
            url_list.append(driver.current_url)
            driver.back()            
        
        
                

IndexError: list index out of range

In [149]:
try:
    driver.find_element_by_link_text('Page suivante').click()
except:

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"link text","selector":"Page suivante"}
  (Session info: chrome=80.0.3987.132)


In [92]:
result_df = criteria_table = pd.read_excel('Data_for_Scraping_Electrician.xlsx',sheet_name='Result')

In [96]:
result_df.head()

,Code Postal,Ville,Téléphone,E-mail,Site Internet,Libellé code NAF 2008,Enseigne,Responsable légal,Activité en clair,Raison Sociale,Dénomination
0,83350,Ramatuelle,33 4 94 97 18 02,tahitibeach@wanadoofr,wwwtahiti-beachcom,Hôtels et hébergement similaire,NaN,Felicia Coppola,NaN,TAHITI BEACH,TAHITI BEACH
1,83000,Toulon,33 4 94 10 88 26,NaN,NaN,Hôtels et hébergement similaire,NaN,M Guenec Alain,Para Hotellerie Location En Meubles Residence ...,GUEN'S MEUBLE,GUEN'S MEUBLE
